In [3]:
import dependency_installer
import xarray as xr
from data_hunter_era5 import DataRetriver
import matplotlib.pyplot as plt


%matplotlib inline
# dependency_installer.main()

In [4]:
import statsmodels.api as sm
import pandas as pd

longitude_value = 44.51420518337152
latitude_value = 40.18315899637762


regression_observed_Data = DataRetriver.retrive_custom_data(
    startDate='2019-11-25',
    endDate='2019-11-28',
    properties= ['10m_u_component_of_wind', '10m_v_component_of_wind','2m_temperature','surface_pressure', 'soil_temperature_level_1'],
    long_min = 43, 
    long_max = 46, 
    lat_min= 38.5,
    lat_max= 41)

# Extract the data for multivariate regression

data_for_regression = xr.merge(regression_observed_Data).sel(longitude=longitude_value, latitude=latitude_value, method='nearest').to_dataframe()

# Drop the 'time' column as it is not needed for regression
# data_for_regression = data_for_regression.drop(columns=['time'])

# Define the target variable and independent variables
target_variable = 't2m'
independent_variables = ['u10', 'v10', 'sp', 'stl1']  # Adjust based on your specific variables

# Extract the target and independent variables from the DataFrame
y = data_for_regression[target_variable]
X = data_for_regression[independent_variables]

# Add a constant term to the independent variables matrix (required for statsmodels)
X = sm.add_constant(X)

# Fit the multivariate regression model
model = sm.OLS(y, X).fit()

# Print the regression summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    t2m   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     68.93
Date:                Sun, 10 Dec 2023   Prob (F-statistic):           1.01e-26
Time:                        18:57:24   Log-Likelihood:                -188.56
No. Observations:                  96   AIC:                             387.1
Df Residuals:                      91   BIC:                             399.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        921.1091    248.618      3.705      0.0